In [1]:
import pandas as pd
import os
import sys 
import importlib

In [2]:
# Subo dos niveles desde data_analysis -> src -> g5-backend
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

In [3]:
from src.paths import RUTA_TITULO
from src.paths import DIR_DATA_PROCESADA
#from src.helpers import leer_tsv_chunks


#! Lo siguiente es para que me tome los cambios realizados mientras actualizo la clase
import src.helpers
importlib.reload(src.helpers)
from src.helpers import DataCleaner

In [4]:
from src.paths import RUTA_PERSONAL
from src.paths import RUTA_CRITICAS
from src.paths import RUTA_PRINCIPALES
from src.paths import RUTA_NOMBRE

Muestro la cantidad y tipos de títulos encontrados

In [5]:
cleaner=DataCleaner(RUTA_TITULO, DIR_DATA_PROCESADA)

In [6]:
lista_tipos=cleaner.valores_unicos_columna('titleType')

print("Tipos de títulos únicos encontrados:")
for tipo in lista_tipos:
    print(f"- {tipo}")

print(f'\n Cantidad total de tipos de títulos: {len(lista_tipos)}')

Tipos de títulos únicos encontrados:
- movie
- short
- tvEpisode
- tvMiniSeries
- tvMovie
- tvPilot
- tvSeries
- tvShort
- tvSpecial
- video
- videoGame

 Cantidad total de tipos de títulos: 11


Filtro 5 títulos de *short, tvMovie, tvShort, tvSpecial*

In [7]:
# Lista de tipos que queremos filtrar
tipos_buscados = ['short', 'tvMovie', 'tvShort', 'tvSpecial']

# Diccionario para almacenar los resultados de cada tipo
resultados = {tipo: [] for tipo in tipos_buscados}

# Leer chunks y filtrar
titulos_iterable = cleaner.leer_tsv_chunks(cleaner.ruta_archivo)

for chunk in titulos_iterable:
    for tipo in tipos_buscados:
        # Si aún no tenemos 5 registros de este tipo, buscamos más
        if len(resultados[tipo]) < 5:
            filtrados = chunk[chunk['titleType'] == tipo]
            # Tomamos solo los que faltan hasta llegar a 5
            resultados[tipo].extend(filtrados.head(5 - len(resultados[tipo])).to_dict('records'))

    # Verificar si ya tenemos 5 de cada tipo
    if all(len(registros) >= 5 for registros in resultados.values()):
        break

# Mostrar resultados con tabla alineada
for tipo in tipos_buscados:
    print(f"\n=== {tipo.upper()} ===")
    header = f"{'Título Original':<35} | {'Año Inicio':<10} | {'Duración':<8} | Géneros"
    print(header)
    print("-" * len(header))

    for registro in resultados[tipo]:
        titulo = registro['originalTitle'][:35]  # Limitar a 35 caracteres
        anio = registro['startYear']
        duracion = registro['runtimeMinutes']
        generos = registro['genres']
        print(f"{titulo:<35} | {anio:<10} | {duracion:<8} | {generos}")


=== SHORT ===
Título Original                     | Año Inicio | Duración | Géneros
---------------------------------------------------------------------
Carmencita                          | 1894       | 1        | Documentary,Short
Le clown et ses chiens              | 1892       | 5        | Animation,Short
Pauvre Pierrot                      | 1892       | 5        | Animation,Comedy,Romance
Un bon bock                         | 1892       | 12       | Animation,Short
Blacksmith Scene                    | 1893       | 1        | Short

=== TVMOVIE ===
Título Original                     | Año Inicio | Duración | Géneros
---------------------------------------------------------------------
Julius Caesar                       | 1938       | 101      | Drama,History
As You Like It                      | 1946       | \N       | Drama
A Midsummer Night's Dream           | 1946       | 150      | Drama,Fantasy
Hamlet Part 1                       | 1947       | 88       | Drama
The Merch

Filtramos sólo las películas.  
De todo el dataset, guarda solo películas y TV movies

In [8]:
# Tipos de películas que queremos mantener
tipos_peliculas = {"titleType": ["movie", "tvMovie"]}

df_peliculas=cleaner.limpiar(filtros=tipos_peliculas)

# Mostrar información general
print(f"Total de registros filtrados: {len(df_peliculas)}")
print("\nDistribución por tipo:")
print(df_peliculas['titleType'].value_counts())

Total de registros filtrados: 876594

Distribución por tipo:
titleType
movie      724491
tvMovie    152103
Name: count, dtype: int64


In [9]:
print("\nPrimeras 5 películas de cada tipo:")
for tipo in tipos_peliculas["titleType"]:
    print(f"\n=== {tipo.upper()} ===")
    print(df_peliculas[df_peliculas['titleType'] == tipo].head()[['primaryTitle', 'startYear', 'runtimeMinutes', 'genres']])


Primeras 5 películas de cada tipo:

=== MOVIE ===
                    primaryTitle startYear runtimeMinutes  \
0                     Miss Jerry      1894             45   
1  The Corbett-Fitzsimmons Fight      1897            100   
2                       Bohemios      1905            100   
3    The Story of the Kelly Gang      1906             70   
4               The Prodigal Son      1907             90   

                       genres  
0                     Romance  
1      Documentary,News,Sport  
2                          \N  
3  Action,Adventure,Biography  
4                       Drama  

=== TVMOVIE ===
                    primaryTitle startYear runtimeMinutes         genres
20302              Julius Caesar      1938            101  Drama,History
26711             As You Like It      1946             \N          Drama
27062  A Midsummer Night's Dream      1946            150  Drama,Fantasy
27632              Hamlet Part 1      1947             88          Drama
27778   

Recorre todas las columnas y muestra cuántos nulos hay en cada una.

In [10]:
# Analizar cada columna
cleaner.mostrar_nulos(df_peliculas)

Análisis de valores nulos por columna:
--------------------------------------------------

Columna: tconst
- Valores '\N': 0
- Valores NaN: 0
- Total nulos: 0
- Porcentaje: 0.00%

Columna: titleType
- Valores '\N': 0
- Valores NaN: 0
- Total nulos: 0
- Porcentaje: 0.00%

Columna: primaryTitle
- Valores '\N': 0
- Valores NaN: 3
- Total nulos: 3
- Porcentaje: 0.00%

Columna: originalTitle
- Valores '\N': 0
- Valores NaN: 3
- Total nulos: 3
- Porcentaje: 0.00%

Columna: isAdult
- Valores '\N': 0
- Valores NaN: 0
- Total nulos: 0
- Porcentaje: 0.00%

Columna: startYear
- Valores '\N': 111,390
- Valores NaN: 0
- Total nulos: 111,390
- Porcentaje: 12.71%

Columna: endYear
- Valores '\N': 876,594
- Valores NaN: 0
- Total nulos: 876,594
- Porcentaje: 100.00%

Columna: runtimeMinutes
- Valores '\N': 314,612
- Valores NaN: 0
- Total nulos: 314,612
- Porcentaje: 35.89%

Columna: genres
- Valores '\N': 90,763
- Valores NaN: 0
- Total nulos: 90,763
- Porcentaje: 10.35%

Registros con al menos un va

Limpieza de nulos.  
Se eliminan los registros donde falten datos en columnas clave.

In [11]:
# Columnas a verificar
columnas_a_filtrar = ['genres', 'primaryTitle', 'originalTitle', 'runtimeMinutes']

# Aplicar el filtro
df_peliculas_limpio = cleaner.limpiar_nulos(df_peliculas, columnas_a_filtrar) 

In [12]:
# Mostrar información sobre los registros filtrados
cleaner.mostrar_registros_filtrados(df_peliculas,df_peliculas_limpio)

print("\nDistribución por tipo después de la limpieza:")
print(df_peliculas_limpio['titleType'].value_counts())

print("\nMuestra de 5 registros limpios:")
df_peliculas_limpio[['primaryTitle', 'titleType', 'startYear', 'runtimeMinutes', 'genres']].head()

Registros originales: 876,594
Registros después de eliminar nulos: 530,367
Registros eliminados: 346,227 (39.50%)

Distribución por tipo después de la limpieza:
titleType
movie      431138
tvMovie     99229
Name: count, dtype: int64

Muestra de 5 registros limpios:


,primaryTitle,titleType,startYear,runtimeMinutes,genres
0,Miss Jerry,movie,1894,45,Romance
1,The Corbett-Fitzsimmons Fight,movie,1897,100,"Documentary,News,Sport"
3,The Story of the Kelly Gang,movie,1906,70,"Action,Adventure,Biography"
4,The Prodigal Son,movie,1907,90,Drama
8,The Fairylogue and Radio-Plays,movie,1908,120,"Adventure,Fantasy"


Filtra solo películas del año 2019 que no sean para adultos.

In [13]:
# Filtrar películas del 2019
filtros={'startYear':'2019',
        'isAdult':0}

df_peliculas_filtrado = cleaner.filtrar_dataframe(df_peliculas_limpio, filtros)

In [14]:
print(f"Total de películas del 2019: {len(df_peliculas_filtrado):,}")
print("\nDistribución por tipo:")
print(df_peliculas_filtrado['titleType'].value_counts())

print("\nEstadísticas de duración (en minutos):")
print(df_peliculas_filtrado['runtimeMinutes'].astype(float).describe())


Total de películas del 2019: 16,810

Distribución por tipo:
titleType
movie      14615
tvMovie     2195
Name: count, dtype: int64

Estadísticas de duración (en minutos):
count    16810.000000
mean        93.533314
std        514.964989
min          1.000000
25%         70.000000
50%         88.000000
75%        101.000000
max      59460.000000
Name: runtimeMinutes, dtype: float64


In [15]:
columnas_a_eliminar=["isAdult"]  #, "endYear"
df_peliculas_filtrado = cleaner.eliminar_columnas(df_peliculas_filtrado,columnas_a_eliminar)

print("\nMuestra de películas del 2019:")
df_peliculas_filtrado.head(10)


Muestra de películas del 2019:


,tconst,titleType,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,genres
87763,tt0116991,movie,Mariette in Ecstasy,Mariette in Ecstasy,2019,\N,101,Drama
217353,tt0385887,movie,Motherless Brooklyn,Motherless Brooklyn,2019,\N,144,"Crime,Drama,Mystery"
233764,tt0437086,movie,Alita: Battle Angel,Alita: Battle Angel,2019,\N,122,"Action,Adventure,Sci-Fi"
235270,tt0441881,movie,Danger Close,Danger Close: The Battle of Long Tan,2019,\N,118,"Action,Drama,War"
237327,tt0448115,movie,Shazam!,Shazam!,2019,\N,132,"Action,Adventure,Comedy"
254146,tt0783640,movie,The Last Full Measure,The Last Full Measure,2019,\N,116,"Drama,War"
254769,tt0791072,movie,Enemigo,Enemigo,2019,\N,93,Drama
255734,tt0800325,movie,The Dirt,The Dirt,2019,\N,107,"Biography,Comedy,Drama"
256662,tt0810836,movie,Dirt Music,Dirt Music,2019,\N,105,"Crime,Drama,Romance"
259568,tt0837563,movie,Pet Sematary,Pet Sematary,2019,\N,100,"Horror,Mystery,Thriller"


In [16]:
cleaner.guardar_csv(df_peliculas_filtrado, "peliculas_2019.csv")

Archivo guardado: peliculas_2019.csv


### Creo archivo para alias (title.akas.tsv)

In [6]:
import src.paths
import src.helpers
importlib.reload(src.paths)
importlib.reload(src.helpers)

<module 'src.helpers' from 'c:\\Users\\melod\\Downloads\\Base de Datos\\TPFinal\\g5-backend\\src\\helpers.py'>

In [7]:
from src.helpers import DatasetManager

In [8]:
from src.paths import RUTA_ALIAS
from src.paths import RUTA_TITULO_2019
from src.paths import RUTA_CRITICAS

In [20]:
dm = DatasetManager()
preview = dm.ver_preview(RUTA_ALIAS, n=4)

Columnas: ['titleId', 'ordering', 'title', 'region', 'language', 'types', 'attributes', 'isOriginalTitle']

Primeras filas:
     titleId  ordering                      title region language  \
0  tt0000001         1                 Carmencita     \N       \N   
1  tt0000001         2                 Carmencita     DE       \N   
2  tt0000001         3                 Carmencita     US       \N   
3  tt0000001         4  Carmencita - spanyol tánc     HU       \N   

         types     attributes  isOriginalTitle  
0     original             \N                1  
1           \N  literal title                0  
2  imdbDisplay             \N                0  
3  imdbDisplay             \N                0  


In [28]:
peliculas_2019 = pd.read_csv(RUTA_TITULO_2019)
print(f"Películas cargadas: {len(peliculas_2019):,}")
peliculas_2019.head()

Películas cargadas: 16,810


,tconst,titleType,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,genres
0,tt0116991,movie,Mariette in Ecstasy,Mariette in Ecstasy,2019,\N,101,Drama
1,tt0385887,movie,Motherless Brooklyn,Motherless Brooklyn,2019,\N,144,"Crime,Drama,Mystery"
2,tt0437086,movie,Alita: Battle Angel,Alita: Battle Angel,2019,\N,122,"Action,Adventure,Sci-Fi"
3,tt0441881,movie,Danger Close,Danger Close: The Battle of Long Tan,2019,\N,118,"Action,Drama,War"
4,tt0448115,movie,Shazam!,Shazam!,2019,\N,132,"Action,Adventure,Comedy"


In [29]:
print(peliculas_2019.columns.tolist())

['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'startYear', 'endYear', 'runtimeMinutes', 'genres']


In [30]:
ids_2019 = set(peliculas_2019['tconst'].astype(str))

print(f"Películas cargadas: {len(peliculas_2019):,}")
print(f"IDs únicos: {len(ids_2019):,}")

cleaner = DataCleaner(RUTA_ALIAS, DIR_DATA_PROCESADA)

Películas cargadas: 16,810
IDs únicos: 16,810


In [54]:
columnas_a_mantener = ["titleId", "title", "isOriginalTitle", "region", "language"]

In [32]:
df_akas = cleaner.limpiar(filtros={"titleId": list(ids_2019)})
print(f"Filtrado akas -> {len(df_akas):,} registros")

Filtrado akas -> 103,980 registros


In [33]:
df_akas = cleaner.mantener_columnas(df_akas, columnas_a_mantener)

In [55]:
cleaner.guardar_csv(df_akas, "alias_2019.csv")

Archivo guardado: alias_2019.csv


In [56]:
ruta_alias = os.path.join(DIR_DATA_PROCESADA, "alias_2019.csv")
alias_2019 = pd.read_csv(ruta_alias)

print(alias_2019.columns.tolist())

['titleId', 'title', 'isOriginalTitle', 'region', 'language']


In [52]:
ids_alias = set(alias_2019["titleId"].astype(str))
ids_peliculas = set(peliculas_2019["tconst"].astype(str))

print(f"Películas cargadas: {len(peliculas_2019):,}")
print(f"IDs únicos en películas: {len(ids_peliculas):,}")
print(f"Registros en alias: {len(alias_2019):,}")
print(f"IDs únicos en alias: {len(ids_alias):,}")

# Intersección (para verificar que coincidan)
ids_en_ambos = ids_peliculas.intersection(ids_alias)
print(f"IDs en ambos datasets: {len(ids_en_ambos):,}")

Películas cargadas: 16,810
IDs únicos en películas: 16,810
Registros en alias: 103,980
IDs únicos en alias: 16,774
IDs en ambos datasets: 16,774


In [53]:
# Porcentaje de películas que tienen al menos un alias
cobertura = (len(ids_en_ambos) / len(ids_peliculas)) * 100
print(f"Cobertura de alias sobre películas: {cobertura:.2f}%")

# Porcentaje de alias que corresponden a alguna película de 2019
cobertura_alias = (len(ids_en_ambos) / len(ids_alias)) * 100
print(f"Cobertura de alias respecto a películas: {cobertura_alias:.2f}%")

Cobertura de alias sobre películas: 99.79%
Cobertura de alias respecto a películas: 100.00%


### Creo archivo para raitings (title.ratings.tsv)

In [80]:
peliculas_2019 = pd.read_csv(RUTA_TITULO_2019)
ids_2019 = set(peliculas_2019["tconst"].astype(str))

In [81]:
cleaner = DataCleaner(RUTA_CRITICAS, DIR_DATA_PROCESADA)

In [82]:
columnas_ratings = ["tconst", "averageRating", "numVotes"]
df_ratings = cleaner.limpiar(filtros={"tconst": list(ids_2019)})

In [83]:
df_ratings = cleaner.mantener_columnas(df_ratings, columnas_ratings)

In [85]:
cleaner.guardar_csv(df_ratings, "criticas_2019.csv")

Archivo guardado: criticas_2019.csv


In [86]:
# Información de verificación
print(f"Registros de Ratings filtrados: {len(df_ratings):,}")
print(f"IDs únicos en Ratings: {df_ratings['tconst'].nunique():,}")

Registros de Ratings filtrados: 11,315
IDs únicos en Ratings: 11,315


### Creo archivo para principals (title.principals.tsv.gz)

In [11]:
from src.paths import RUTA_PRINCIPALES
from src.paths import RUTA_TITULO_2019

from src.helpers import DatasetManager

In [12]:
peliculas_2019 = pd.read_csv(RUTA_TITULO_2019)
ids_2019 = set(peliculas_2019["tconst"].astype(str))

In [13]:
print(f"Películas cargadas: {len(peliculas_2019):,}")
print(f"IDs únicos: {len(ids_2019):,}")

Películas cargadas: 16,810
IDs únicos: 16,810


In [91]:
cleaner = DataCleaner(RUTA_PRINCIPALES, DIR_DATA_PROCESADA)

In [15]:
dm = DatasetManager()
preview = dm.ver_preview(RUTA_PRINCIPALES, n=4)

Columnas: ['tconst', 'ordering', 'nconst', 'category', 'job', 'characters']

Primeras filas:
      tconst  ordering     nconst         category                      job  \
0  tt0000001         1  nm1588970             self                       \N   
1  tt0000001         2  nm0005690         director                       \N   
2  tt0000001         3  nm0005690         producer                 producer   
3  tt0000001         4  nm0374658  cinematographer  director of photography   

  characters  
0   ["Self"]  
1         \N  
2         \N  
3         \N  


In [16]:
columnas_a_mantener = ["tconst", "nconst", "category", "job", "characters"]
listado_trabajos=["actor", "actress", "self", "director", "writer","composer","producer"]

In [17]:
df_principals = cleaner.limpiar(filtros={"tconst": list(ids_2019),
                                         "category":listado_trabajos})
print(f"Filtrado principals -> {len(df_principals):,} registros")

Filtrado principals -> 185,915 registros


In [93]:
trabajos_unicos = set(df_principals["category"].dropna())
print("Tipos de Trabajos únicos encontrados:")
for tipo in trabajos_unicos:
    print(f"- {tipo}")

print(f'\n Cantidad total de tipos de títulos: {len(trabajos_unicos)}')

Tipos de Trabajos únicos encontrados:
- actress
- composer
- self
- director
- producer
- actor
- writer

 Cantidad total de tipos de títulos: 7


In [19]:
print(f"Registros de Principals filtrados: {len(df_principals):,}")
print(f"IDs únicos en Principals: {df_principals['tconst'].nunique():,}")

Registros de Principals filtrados: 185,915
IDs únicos en Principals: 16,650


In [20]:
df_principals = cleaner.mantener_columnas(df_principals, columnas_a_mantener)

In [26]:
cleaner.guardar_csv(df_principals, "principales_2019.csv")

Archivo guardado: principales_2019.csv


In [23]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "./.")))
from src.paths import DIR_DATA_PROCESADA

In [25]:
RUTA_PRINCIPALS_2019 =os.path.join(DIR_DATA_PROCESADA,"principals_2019.csv")
dm = DatasetManager()
preview = dm.ver_preview(RUTA_PRINCIPALS_2019, n=4)

Columnas: ['tconst,nconst,category,job,characters']

Primeras filas:
               tconst,nconst,category,job,characters
0  tt0116991,nm0642374,actress,\N,"[""Mariette Ba...
1  tt0116991,nm0001693,actress,\N,"[""Mother Sain...
2  tt0116991,nm0032185,actress,\N,"[""Sister Phil...
3      tt0116991,nm0064229,actress,\N,"[""Sister""]"


### Creo archivo para names (name.basics.tsv.gz)

In [34]:
import src.paths
import src.helpers

In [83]:
importlib.reload(src.paths)
importlib.reload(src.helpers)

<module 'src.helpers' from 'c:\\Users\\melod\\Downloads\\Base de Datos\\TPFinal\\g5-backend\\src\\helpers.py'>

In [94]:
from src.paths import RUTA_NOMBRE
from src.paths import RUTA_PRINCIPALES_2019
from src.paths import RUTA_TITULO_2019

from src.helpers import DatasetManager
from src.helpers import DataCleaner

In [86]:
dm = DatasetManager()
preview = dm.ver_preview(RUTA_NOMBRE, n=4)

Columnas: ['nconst', 'primaryName', 'birthYear', 'deathYear', 'primaryProfession', 'knownForTitles']

Primeras filas:
      nconst      primaryName  birthYear deathYear  \
0  nm0000001     Fred Astaire       1899      1987   
1  nm0000002    Lauren Bacall       1924      2014   
2  nm0000003  Brigitte Bardot       1934        \N   
3  nm0000004     John Belushi       1949      1982   

                    primaryProfession                           knownForTitles  
0        actor,miscellaneous,producer  tt0072308,tt0050419,tt0027125,tt0025164  
1  actress,soundtrack,archive_footage  tt0037382,tt0075213,tt0038355,tt0117057  
2   actress,music_department,producer  tt0057345,tt0049189,tt0056404,tt0054452  
3       actor,writer,music_department  tt0072562,tt0077975,tt0080455,tt0078723  


In [95]:
df_principals = pd.read_csv(RUTA_PRINCIPALES_2019)  #de aca saco los "nconst"

In [96]:
cleaner = DataCleaner(RUTA_NOMBRE , DIR_DATA_PROCESADA)

In [98]:
columnas_a_mantener = ['nconst', 'primaryName', 'birthYear','primaryProfession']#, 'knownForTitles'

In [99]:
nconsts_relevantes = set(df_principals["nconst"].astype(str))
print(f"Personas únicas en principals: {len(nconsts_relevantes):,}")

Personas únicas en principals: 133,961


In [100]:
df_names = cleaner.limpiar(filtros={'nconst': list(nconsts_relevantes)})

In [101]:
print(f"Registros de names filtrados: {len(df_names):,}")
print(f"IDs únicos en names: {df_names['nconst'].nunique():,}")

Registros de names filtrados: 133,957
IDs únicos en names: 133,957


In [102]:
df_names = cleaner.mantener_columnas(df_names,columnas_a_mantener)

In [103]:
cleaner.guardar_csv(df_names, "nombres_2019.csv")

Archivo guardado: nombres_2019.csv


In [72]:
from src.paths import RUTA_NOMBRE_2019

In [ ]:
dm = DatasetManager()
preview = dm.ver_preview(RUTA_NOMBRE_2019, n=4)